In [1]:
import pandas as pd
df = pd.read_excel('/Volumes/transcend/大檔案/中華郵政/雷達圖指標/雷達指標表.xlsx')

In [2]:
df.head()

,ACC01,局名,鄉鎮市區,區域人口數,附近郵局數,獲利能力,案件數量,特約客戶數量,營業天數,空值量,交寄時間異常
0,100001,臺北東門,中正區,158709,8,1700817,122151,27,71,145277,693
1,100005,臺北南海,中正區,158709,9,2577493,555857,30,71,215173,0
2,100006,臺北螢橋,中正區,158709,4,989057,53165,18,60,75189,13
3,100008,臺北復興橋,中正區,158709,11,1095432,176533,25,60,102501,10
4,100012,行政院,中正區,158709,9,641832,131994,17,60,61454,0


In [97]:
df2 = df.copy()
df2 = df2[['ACC01','局名']]

In [98]:
df2['吸金能力'] = df['獲利能力']
df2['服務能力'] = df['案件數量'] / df['營業天數']
df2['資料完整度'] = 1 / (df['空值量']*0.7 + df['交寄時間異常']*0.3)
df2['招商能力'] = df['特約客戶數量']
df3 = df2.copy()

In [103]:
# 區域影響力
df_area = pd.DataFrame()
df2['鄉鎮市區'] = df['鄉鎮市區']
for i in df2['鄉鎮市區'].value_counts().index:
    score = []
    for k in range(1,len(df2[df2['鄉鎮市區']==i])+1):
        score.append(k/len(df2[df2['鄉鎮市區']==i]))
    df_temp = pd.concat([df2[df2['鄉鎮市區']==i].sort_values('吸金能力', ascending=True).reset_index(drop=True), pd.DataFrame(score, columns=['score1'])], axis=1)
    df_temp = pd.concat([df_temp.sort_values('服務能力', ascending=True).reset_index(drop=True), pd.DataFrame(score, columns=['score2'])], axis=1)
    df_temp = pd.concat([df_temp.sort_values('資料完整度', ascending=True).reset_index(drop=True), pd.DataFrame(score, columns=['score3'])], axis=1)
    df_temp = pd.concat([df_temp.sort_values('招商能力', ascending=True).reset_index(drop=True), pd.DataFrame(score, columns=['score4'])], axis=1)
    df_temp['區域影響力'] = df_temp['score1'] + df_temp['score2'] + df_temp['score3'] + df_temp['score4']
    df_area = pd.concat([df_area, df_temp[['ACC01','區域影響力']]])
df_area = df_area.reset_index(drop=True)
df2 = pd.merge(df2, df_area, on=['ACC01'], how='left')

### 標準化

In [105]:
from sklearn import preprocessing
minmax = preprocessing.MinMaxScaler()
df3[['吸金能力','服務能力','資料完整度','區域影響力','招商能力']] = minmax.fit_transform(df2[['吸金能力','服務能力','資料完整度','區域影響力','招商能力']])
df3['總分'] = df3['吸金能力'] + df3['服務能力'] + df3['資料完整度'] + df3['區域影響力'] + df3['招商能力']
# df3['總分'] = df3['吸金能力'] + df3['區域影響力'] + df3['招商能力']

/Users/andy/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [107]:
# df3 = df3.drop('鄉鎮市區', axis=1)
df3.sort_values('總分', ascending=False).head(20)

,ACC01,局名,吸金能力,服務能力,資料完整度,招商能力,區域影響力,總分
45,100091,臺北金南,0.211907,1.000000,0.000991,0.922581,0.478261,2.613740
132,100148,內湖,1.000000,0.009137,0.000617,0.853763,0.678930,2.542447
242,220071,中和,0.700910,0.010085,0.001718,1.000000,0.652174,2.364888
176,220000,板橋文化路,0.760990,0.006025,0.001929,0.582796,0.652174,2.003914
298,241036,新莊,0.471690,0.022982,0.004295,0.634409,0.675362,1.808738
168,200039,汐止,0.367773,0.058799,0.004004,0.651613,0.695652,1.777842
276,241056,林口,0.544425,0.098806,0.002731,0.384946,0.721739,1.752647
150,100167,南港,0.478006,0.005359,0.004545,0.524731,0.690821,1.703462
47,100108,臺北敦南,0.903784,0.003661,0.000816,0.223656,0.565217,1.697134
213,220041,新店,0.336109,0.029523,0.000000,0.546237,0.668737,1.580607


### 標準化2

In [142]:
score = []
for i in range(1,319):
    score.append(i/318)

In [150]:
df4 = pd.concat([df2.sort_values('吸金能力', ascending=True).reset_index(drop=True), pd.DataFrame(score, columns=['score1'])], axis=1)
df4 = pd.concat([df4.sort_values('服務能力', ascending=True).reset_index(drop=True), pd.DataFrame(score, columns=['score2'])], axis=1)
df4 = pd.concat([df4.sort_values('資料完整度', ascending=True).reset_index(drop=True), pd.DataFrame(score, columns=['score3'])], axis=1)
df4 = pd.concat([df4.sort_values('招商能力', ascending=True).reset_index(drop=True), pd.DataFrame(score, columns=['score4'])], axis=1)
df4 = pd.concat([df4.sort_values('區域影響力', ascending=True).reset_index(drop=True), pd.DataFrame(score, columns=['score5'])], axis=1)
df4 = df4[['ACC01','局名','score1','score2','score3','score4','score5']]
df4 = df4.rename(columns={'score1': '吸金能力', 'score2': '服務能力', 'score3': '資料完整度', 'score4': '招商能力', 'score5': '區域影響力'})
df4['總分'] = df4['吸金能力'] + df4['服務能力'] + df4['資料完整度'] + df4['招商能力'] + df4['區域影響力']
df4 = df4.sort_values('ACC01', ascending=True).reset_index(drop=True)

In [151]:
df4.sort_values('總分', ascending=False)

,ACC01,局名,吸金能力,服務能力,資料完整度,招商能力,區域影響力,總分
316,241056,林口,0.965409,0.996855,0.037736,0.965409,0.959119,3.924528
296,241036,新莊,0.949686,0.984277,0.062893,0.987421,0.924528,3.908805
168,200039,汐止,0.911950,0.993711,0.059748,0.990566,0.943396,3.899371
122,100148,內湖,1.000000,0.968553,0.006289,0.993711,0.927673,3.896226
242,220071,中和,0.990566,0.971698,0.015723,1.000000,0.905660,3.883648
140,100167,南港,0.952830,0.937107,0.066038,0.974843,0.940252,3.871069
236,220064,永和,0.918239,0.899371,0.119497,0.962264,0.952830,3.852201
253,220082,樹林,0.839623,0.943396,0.144654,0.971698,0.937107,3.836478
176,220000,板橋文化路,0.993711,0.946541,0.018868,0.984277,0.877358,3.820755
188,220014,板橋國慶,0.805031,0.987421,0.194969,0.952830,0.864780,3.805031


### 標準化1+2

In [153]:
df5 = df3.copy()
df5['吸金能力'] = (df3['吸金能力'] + df4['吸金能力']) / 2
df5['服務能力'] = (df3['服務能力'] + df4['服務能力']) / 2
df5['資料完整度'] = (df3['資料完整度'] + df4['資料完整度']) / 2
df5['招商能力'] = (df3['招商能力'] + df4['招商能力']) / 2
df5['區域影響力'] = (df3['區域影響力'] + df4['區域影響力']) / 2
df5['總分'] = df5['吸金能力'] + df5['服務能力'] + df5['資料完整度'] + df5['招商能力'] + df5['區域影響力']

In [154]:
df5.sort_values('總分', ascending=False)

,ACC01,局名,吸金能力,服務能力,資料完整度,招商能力,區域影響力,總分
242,220071,中和,0.845738,0.490892,0.008721,1.000000,0.778917,3.124268
176,220000,板橋文化路,0.877350,0.476283,0.010399,0.783536,0.764766,2.912335
168,200039,汐止,0.639862,0.526255,0.031876,0.821089,0.819524,2.838607
276,241056,林口,0.696741,0.439340,0.079982,0.664171,0.835712,2.715946
213,220041,新店,0.619313,0.510045,0.001572,0.763684,0.793488,2.688102
236,220064,永和,0.644547,0.451489,0.063523,0.654250,0.831488,2.645297
45,100091,臺北金南,0.417274,0.674528,0.215904,0.782045,0.509571,2.599323
244,220073,中和中山路,0.608660,0.492718,0.052062,0.619923,0.761622,2.534984
296,241033,三芝,0.517650,0.492453,0.049438,0.500162,0.962264,2.521967
71,100036,臺北光復,0.730253,0.428068,0.062544,0.524346,0.730175,2.475385
